# SQL Project
You were hired by Ironhack to perform an Analytics Consulting Project entitled: competitive landscape.

Your mission is to create and populate an appropriate database with many coding schools that are our competition, as well as design an suitable queries that answer business questions of interest (to be defined by you)


**Suggested Steps in the Project:**


*   Read this notebook and understand each function. Comment the code appropriately

*   Populate the list of schools with a wider variety of schools (how are you going to get the school ID?)

* Take a look at the obtained dataframes. What dimensions do you have? what keys do you have? how could the different dataframes be connected?

* Go back to the drawing board and try to create an entity relationship diagram for tables available

* Once you have the schemas you want, you will need to:
  - create the suitable SQL queries to create the tables and populate them
  - run these queries using the appropriate Python connectors
  
* Bonus: How will this datamodel be updated in the future? Please write auxiliary functions that test the database for data quality issues. For example: how could you make sure you only include the most recent comments when you re-run the script?


# Suggested Deliverables

* 5-6 minute presentation of data model created, decision process and business analysis proposed

* exported .sql file with the final schema

* Supporting python files used to generate all logic

* High level documentation explaining tables designed and focusing on update methods

Crucial hint: check out the following tutorial:
https://www.dataquest.io/blog/sql-insert-tutorial/


In [1]:
# you must populate this dict with the schools required -> try talking to the teaching team about this


schools = {   
'ironhack' : 10828,
'app-academy' : 10525,
'springboard' : 11035    
}

import re
import pandas as pd
from pandas.io.json import json_normalize
import requests



def get_comments_school(school):
    TAG_RE = re.compile(r'<[^>]+>')
    # defines url to make api call to data -> dynamic with school if you want to scrape competition
    url = "https://www.switchup.org/chimera/v1/school-review-list?mainTemplate=school-review-list&path=%2Fbootcamps%2F" + school + "&isDataTarget=false&page=3&perPage=10000&simpleHtml=true&truncationLength=250"
    #makes get request and converts answer to json
    # url defines the page of all the information, request is made, and information is returned to data variable
    data = requests.get(url).json()
    #converts json to dataframe
    reviews =  pd.DataFrame(data['content']['reviews'])
  
    #aux function to apply regex and remove tags
    def remove_tags(x):
        return TAG_RE.sub('',x)
    reviews['review_body'] = reviews['body'].apply(remove_tags)
    reviews['school'] = school
    return reviews

In [2]:
# could you write this as a list comprehension? ;)
comments = []

for school in schools.keys():
    print(school)
    comments.append(get_comments_school(school))

comments = pd.concat(comments)

ironhack
app-academy
springboard


In [3]:
comments

,id,name,anonymous,hostProgramName,graduatingYear,isAlumni,jobTitle,tagline,body,rawBody,...,queryDate,program,user,overallScore,comments,overall,curriculum,jobSupport,review_body,school
0,306372,Sergio Burgos,False,None,2023.0,False,International Negotiator,The Most Intense Academic Challenge,"<span class=""truncatable""><p></p><p>After comp...",<p>After completing my Data Analytics Bootcamp...,...,2023-11-10,Data Analytics Bootcamp,{'image': None},3.3,[],3.0,3.0,4.0,After completing my Data Analytics Bootcamp wi...,ironhack
1,306215,Anonymous,True,None,2023.0,True,,Transformative Experience: My Time at Ironhack,"<span class=""truncatable""><p></p><p>Pros: 1)In...",<p>Pros: 1)Intensive Learning 2)Real-World Pro...,...,2023-11-06,Web Development Bootcamp,{'image': None},4.0,[],4.0,4.0,4.0,Pros: 1)Intensive Learning 2)Real-World Projec...,ironhack
2,306068,Anonymous,True,None,2023.0,False,Full stack development,Now I can do it,"<span class=""truncatable""><p></p><p>7 months a...","<p>7 months ago, I only had an idea about html...",...,2023-10-31,None,{'image': None},5.0,[],5.0,5.0,5.0,"7 months ago, I only had an idea about html an...",ironhack
3,305297,Utku Cikmaz,False,None,2023.0,False,Full Stack Web Developer,It was good,"<span class=""truncatable""><p></p><p>The course...","<p>The course was great. Especially, Luis is a...",...,2023-10-02,Web Development Bootcamp,{'image': None},4.0,[],5.0,3.0,4.0,"The course was great. Especially, Luis is a gr...",ironhack
4,305278,Nirmal Hodge,False,None,2023.0,False,Product Designer,Ironhack 100% Worth It!,"<span class=""truncatable""><p></p><p>I joined t...",<p>I joined the UX/ UI Bootcamp and to be hone...,...,2023-09-30,UX/UI Design Bootcamp,{'image': None},5.0,[],5.0,5.0,5.0,I joined the UX/ UI Bootcamp and to be honest ...,ironhack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1560,234894,Stephanie S.,False,None,2015.0,True,Website Manager-UX Lead,Excellent and comprehensive UX course,"<span class=""truncatable""><p>The UX Design Wor...",The UX Design Workshop gave me a great underst...,...,2016-12-15,UX Design,{'image': None},5.0,[],5.0,5.0,5.0,The UX Design Workshop gave me a great underst...,springboard
1561,234877,Joe Fang,False,None,2015.0,True,UX Designer,Springboard helped me start my career in UX,"<span class=""truncatable""><p>Just some backgro...","Just some background information on myself, I ...",...,2016-12-15,UX Design,{'image': None},5.0,[],5.0,5.0,5.0,"Just some background information on myself, I ...",springboard
1562,234838,Zeina,False,None,2016.0,True,Senior Graphic Designer,Highly recommended,"<span class=""truncatable""><p>I enjoyed the pro...","I enjoyed the program very much, gained a lot ...",...,2016-12-15,UX Design,{'image': None},3.7,[],4.0,4.0,3.0,"I enjoyed the program very much, gained a lot ...",springboard
1563,242681,Jean,False,None,2016.0,True,Lead Designer/Creative Director,A way to stay current for the highly motivated!,"<span class=""truncatable""><p>This was my first...",This was my first online experience and it did...,...,2016-12-12,UX Design,{'image': None},5.0,[],5.0,5.0,5.0,This was my first online experience and it did...,springboard


In [4]:
from pandas.io.json import json_normalize

def get_school_info(school, school_id):
    url = 'https://www.switchup.org/chimera/v1/bootcamp-data?mainTemplate=bootcamp-data%2Fdescription&path=%2Fbootcamps%2F'+ str(school) + '&isDataTarget=false&bootcampId='+ str(school_id) + '&logoTag=logo&truncationLength=250&readMoreOmission=...&readMoreText=Read%20More&readLessText=Read%20Less'

    data = requests.get(url).json()

    data.keys()

    courses = data['content']['courses']
    courses_df = pd.DataFrame(courses, columns= ['courses'])

    locations = data['content']['locations']
    locations_df = json_normalize(locations)

    badges_df = pd.DataFrame(data['content']['meritBadges'])
    
    website = data['content']['webaddr']
    description = data['content']['description']
    logoUrl = data['content']['logoUrl']
    school_df = pd.DataFrame([website,description,logoUrl]).T
    school_df.columns =  ['website','description','LogoUrl']

    locations_df['school'] = school
    courses_df['school'] = school
    badges_df['school'] = school
    school_df['school'] = school
    

    locations_df['school_id'] = school_id
    courses_df['school_id'] = school_id
    badges_df['school_id'] = school_id
    school_df['school_id'] = school_id

    return locations_df, courses_df, badges_df, school_df

locations_list = []
courses_list = []
badges_list = []
schools_list = []

for school, id in schools.items():
    print(school)
    a,b,c,d = get_school_info(school,id)
    
    locations_list.append(a)
    courses_list.append(b)
    badges_list.append(c)
    schools_list.append(d)



ironhack


C:\Users\Win10 Pro x64\AppData\Local\Temp\ipykernel_14148\1181667300.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  locations_df = json_normalize(locations)


app-academy


C:\Users\Win10 Pro x64\AppData\Local\Temp\ipykernel_14148\1181667300.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  locations_df = json_normalize(locations)


springboard


C:\Users\Win10 Pro x64\AppData\Local\Temp\ipykernel_14148\1181667300.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  locations_df = json_normalize(locations)


In [5]:
locations_list

[      id               description  country.id   country.name country.abbrev  \
 0  15901           Berlin, Germany        57.0        Germany             DE   
 1  16022       Mexico City, Mexico        29.0         Mexico             MX   
 2  16086    Amsterdam, Netherlands        59.0    Netherlands             NL   
 3  16088         Sao Paulo, Brazil        42.0         Brazil             BR   
 4  16109             Paris, France        38.0         France             FR   
 5  16375  Miami, FL, United States         1.0  United States             US   
 6  16376             Madrid, Spain        12.0          Spain             ES   
 7  16377          Barcelona, Spain        12.0          Spain             ES   
 8  16709          Lisbon, Portugal        28.0       Portugal             PT   
 9  17233                    Online         NaN            NaN            NaN   
 
    city.id    city.name city.keyword  state.id state.name state.abbrev  \
 0  31156.0       Berlin       b

In [6]:
locations = pd.concat(locations_list)
locations

,id,description,country.id,country.name,country.abbrev,city.id,city.name,city.keyword,state.id,state.name,state.abbrev,state.keyword,school,school_id
0,15901,"Berlin, Germany",57.0,Germany,DE,31156.0,Berlin,berlin,NaN,NaN,NaN,NaN,ironhack,10828
1,16022,"Mexico City, Mexico",29.0,Mexico,MX,31175.0,Mexico City,mexico-city,NaN,NaN,NaN,NaN,ironhack,10828
2,16086,"Amsterdam, Netherlands",59.0,Netherlands,NL,31168.0,Amsterdam,amsterdam,NaN,NaN,NaN,NaN,ironhack,10828
3,16088,"Sao Paulo, Brazil",42.0,Brazil,BR,31121.0,Sao Paulo,sao-paulo,NaN,NaN,NaN,NaN,ironhack,10828
4,16109,"Paris, France",38.0,France,FR,31136.0,Paris,paris,NaN,NaN,NaN,NaN,ironhack,10828
5,16375,"Miami, FL, United States",1.0,United States,US,31.0,Miami,miami,11.0,Florida,FL,florida,ironhack,10828
6,16376,"Madrid, Spain",12.0,Spain,ES,31052.0,Madrid,madrid,NaN,NaN,NaN,NaN,ironhack,10828
7,16377,"Barcelona, Spain",12.0,Spain,ES,31170.0,Barcelona,barcelona,NaN,NaN,NaN,NaN,ironhack,10828
8,16709,"Lisbon, Portugal",28.0,Portugal,PT,31075.0,Lisbon,lisbon,NaN,NaN,NaN,NaN,ironhack,10828
9,17233,Online,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Online,Online,online,ironhack,10828


In [7]:
courses = pd.concat(courses_list)
courses.head(10)

,courses,school,school_id
0,Cyber Security Bootcamp,ironhack,10828
1,Cybersecurity Part-Time,ironhack,10828
2,Data Analytics Bootcamp,ironhack,10828
3,Data Analytics Part-Time,ironhack,10828
4,UX/UI Design Bootcamp,ironhack,10828
5,UX/UI Design Part-Time,ironhack,10828
6,Web Development Bootcamp,ironhack,10828
7,Web Development Part-Time,ironhack,10828
0,16-week Software Engineering Program,app-academy,10525
1,Bootcamp Prep,app-academy,10525


In [8]:
badges = pd.concat(badges_list)
badges


,name,keyword,description,school,school_id
0,Available Online,available_online,<p>School offers fully online courses</p>,ironhack,10828
1,Verified Outcomes,verified_outcomes,<p>School publishes a third-party verified out...,ironhack,10828
2,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,ironhack,10828
0,Available Online,available_online,<p>School offers fully online courses</p>,app-academy,10525
1,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,app-academy,10525
2,Job Guarantee,job_guarantee,<p>School guarantees job placement</p>,app-academy,10525
0,Available Online,available_online,<p>School offers fully online courses</p>,springboard,11035
1,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,springboard,11035
2,Job Guarantee,job_guarantee,<p>School guarantees job placement</p>,springboard,11035


In [9]:
# any data cleaning still missing here? take a look at the description
schools = pd.concat(schools_list)
schools.head()

,website,description,LogoUrl,school,school_id
0,www.ironhack.com/en,"<span class=""truncatable""><p>Ironhack is a glo...",https://d92mrp7hetgfk.cloudfront.net/images/si...,ironhack,10828
0,appacademy.io,"<span class=""truncatable""><p>Founded in 2012, ...",https://d92mrp7hetgfk.cloudfront.net/images/si...,app-academy,10525
0,www.springboard.com/?utm_source=switchup&utm_m...,"<span class=""truncatable""><p>Springboard is an...",https://d92mrp7hetgfk.cloudfront.net/images/si...,springboard,11035


In [10]:
locations

,id,description,country.id,country.name,country.abbrev,city.id,city.name,city.keyword,state.id,state.name,state.abbrev,state.keyword,school,school_id
0,15901,"Berlin, Germany",57.0,Germany,DE,31156.0,Berlin,berlin,NaN,NaN,NaN,NaN,ironhack,10828
1,16022,"Mexico City, Mexico",29.0,Mexico,MX,31175.0,Mexico City,mexico-city,NaN,NaN,NaN,NaN,ironhack,10828
2,16086,"Amsterdam, Netherlands",59.0,Netherlands,NL,31168.0,Amsterdam,amsterdam,NaN,NaN,NaN,NaN,ironhack,10828
3,16088,"Sao Paulo, Brazil",42.0,Brazil,BR,31121.0,Sao Paulo,sao-paulo,NaN,NaN,NaN,NaN,ironhack,10828
4,16109,"Paris, France",38.0,France,FR,31136.0,Paris,paris,NaN,NaN,NaN,NaN,ironhack,10828
5,16375,"Miami, FL, United States",1.0,United States,US,31.0,Miami,miami,11.0,Florida,FL,florida,ironhack,10828
6,16376,"Madrid, Spain",12.0,Spain,ES,31052.0,Madrid,madrid,NaN,NaN,NaN,NaN,ironhack,10828
7,16377,"Barcelona, Spain",12.0,Spain,ES,31170.0,Barcelona,barcelona,NaN,NaN,NaN,NaN,ironhack,10828
8,16709,"Lisbon, Portugal",28.0,Portugal,PT,31075.0,Lisbon,lisbon,NaN,NaN,NaN,NaN,ironhack,10828
9,17233,Online,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Online,Online,online,ironhack,10828


# Locations DF Data Cleaning
 

In [11]:
locations.columns


Index(['id', 'description', 'country.id', 'country.name', 'country.abbrev',
       'city.id', 'city.name', 'city.keyword', 'state.id', 'state.name',
       'state.abbrev', 'state.keyword', 'school', 'school_id'],
      dtype='object')

In [12]:
locations.drop(columns=['country.id','country.abbrev', 'city.id','city.keyword','state.id', 'state.name', 'state.abbrev', 'state.keyword'], inplace=True)
locations

,id,description,country.name,city.name,school,school_id
0,15901,"Berlin, Germany",Germany,Berlin,ironhack,10828
1,16022,"Mexico City, Mexico",Mexico,Mexico City,ironhack,10828
2,16086,"Amsterdam, Netherlands",Netherlands,Amsterdam,ironhack,10828
3,16088,"Sao Paulo, Brazil",Brazil,Sao Paulo,ironhack,10828
4,16109,"Paris, France",France,Paris,ironhack,10828
5,16375,"Miami, FL, United States",United States,Miami,ironhack,10828
6,16376,"Madrid, Spain",Spain,Madrid,ironhack,10828
7,16377,"Barcelona, Spain",Spain,Barcelona,ironhack,10828
8,16709,"Lisbon, Portugal",Portugal,Lisbon,ironhack,10828
9,17233,Online,NaN,NaN,ironhack,10828


  Renaming columns to match the column names in the MySQL DB

In [13]:
locations.rename(columns={"id": "location_id"}, inplace=True)

In [14]:
locations.columns = locations.columns.str.replace(".", "_")
locations

C:\Users\Win10 Pro x64\AppData\Local\Temp\ipykernel_14148\870817422.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  locations.columns = locations.columns.str.replace(".", "_")


,location_id,description,country_name,city_name,school,school_id
0,15901,"Berlin, Germany",Germany,Berlin,ironhack,10828
1,16022,"Mexico City, Mexico",Mexico,Mexico City,ironhack,10828
2,16086,"Amsterdam, Netherlands",Netherlands,Amsterdam,ironhack,10828
3,16088,"Sao Paulo, Brazil",Brazil,Sao Paulo,ironhack,10828
4,16109,"Paris, France",France,Paris,ironhack,10828
5,16375,"Miami, FL, United States",United States,Miami,ironhack,10828
6,16376,"Madrid, Spain",Spain,Madrid,ironhack,10828
7,16377,"Barcelona, Spain",Spain,Barcelona,ironhack,10828
8,16709,"Lisbon, Portugal",Portugal,Lisbon,ironhack,10828
9,17233,Online,NaN,NaN,ironhack,10828


Replacing missing values with 'Online', because all of the missing values are in the rows with online courses

In [15]:
locations.fillna("Online", inplace=True)
locations

,location_id,description,country_name,city_name,school,school_id
0,15901,"Berlin, Germany",Germany,Berlin,ironhack,10828
1,16022,"Mexico City, Mexico",Mexico,Mexico City,ironhack,10828
2,16086,"Amsterdam, Netherlands",Netherlands,Amsterdam,ironhack,10828
3,16088,"Sao Paulo, Brazil",Brazil,Sao Paulo,ironhack,10828
4,16109,"Paris, France",France,Paris,ironhack,10828
5,16375,"Miami, FL, United States",United States,Miami,ironhack,10828
6,16376,"Madrid, Spain",Spain,Madrid,ironhack,10828
7,16377,"Barcelona, Spain",Spain,Barcelona,ironhack,10828
8,16709,"Lisbon, Portugal",Portugal,Lisbon,ironhack,10828
9,17233,Online,Online,Online,ironhack,10828


Saving table as .csv file

In [16]:
locations.to_csv("CSV//locations.csv",index = False, sep=";")

Data cleaning schools DF

In [17]:
schools

,website,description,LogoUrl,school,school_id
0,www.ironhack.com/en,"<span class=""truncatable""><p>Ironhack is a glo...",https://d92mrp7hetgfk.cloudfront.net/images/si...,ironhack,10828
0,appacademy.io,"<span class=""truncatable""><p>Founded in 2012, ...",https://d92mrp7hetgfk.cloudfront.net/images/si...,app-academy,10525
0,www.springboard.com/?utm_source=switchup&utm_m...,"<span class=""truncatable""><p>Springboard is an...",https://d92mrp7hetgfk.cloudfront.net/images/si...,springboard,11035


In [18]:
schools = schools[['school_id','school','website', 'LogoUrl', 'description']].copy()
schools['description'] = schools['description'].str.replace(r'(?<=<).*?(?=>)', '', regex = True)
schools['description'] = schools['description'].str.replace('<>', '', regex = True)
schools['description'] = schools['description'].str.replace('Read Less|Read More', '', regex = True)
#schools.reset_index(inplace = True, drop = True)
#schools.index += 1

schools.head()

,school_id,school,website,LogoUrl,description
0,10828,ironhack,www.ironhack.com/en,https://d92mrp7hetgfk.cloudfront.net/images/si...,Ironhack is a global tech school with 9 campus...
0,10525,app-academy,appacademy.io,https://d92mrp7hetgfk.cloudfront.net/images/si...,"Founded in 2012, App Academy is a world-renown..."
0,11035,springboard,www.springboard.com/?utm_source=switchup&utm_m...,https://d92mrp7hetgfk.cloudfront.net/images/si...,Springboard is an online learning platform tha...


In [19]:
schools.to_csv("CSV//schools.csv",index = False, sep=";")

Data cleaning courses DF

In [20]:
courses.head(3)

,courses,school,school_id
0,Cyber Security Bootcamp,ironhack,10828
1,Cybersecurity Part-Time,ironhack,10828
2,Data Analytics Bootcamp,ironhack,10828


In [21]:
column_replacements = {
    'courses': 'course'
}
# Using lambda function we can replace the column names: 
courses.rename(columns=lambda x: x.replace('courses', 'course'), inplace=True)
courses.rename(columns=column_replacements, inplace=True)

courses.head(3)

,course,school,school_id
0,Cyber Security Bootcamp,ironhack,10828
1,Cybersecurity Part-Time,ironhack,10828
2,Data Analytics Bootcamp,ironhack,10828


In [22]:
courses.drop(columns='school', inplace=True)

Resetting Index so that we get unique values for each program and copying those in a new column 

In [23]:
courses = courses[['course', 'school_id']].copy()
courses.reset_index(drop=True, inplace=True)
courses['index'] = courses.index + 1

courses

,course,school_id,index
0,Cyber Security Bootcamp,10828,1
1,Cybersecurity Part-Time,10828,2
2,Data Analytics Bootcamp,10828,3
3,Data Analytics Part-Time,10828,4
4,UX/UI Design Bootcamp,10828,5
5,UX/UI Design Part-Time,10828,6
6,Web Development Bootcamp,10828,7
7,Web Development Part-Time,10828,8
8,16-week Software Engineering Program,10525,9
9,Bootcamp Prep,10525,10


Renaming 'Index' column to 'course_id'

In [24]:
courses.rename(columns={'index': 'course_id'}, inplace=True)
courses = courses.reindex(columns=['course_id', 'course', 'school_id'])


courses.head(3)

,course_id,course,school_id
0,1,Cyber Security Bootcamp,10828
1,2,Cybersecurity Part-Time,10828
2,3,Data Analytics Bootcamp,10828


In [25]:
courses.to_csv("CSV//courses.csv",index = False, sep=";")

Data Cleaning Badges DF

In [27]:
badges

,name,keyword,description,school,school_id
0,Available Online,available_online,<p>School offers fully online courses</p>,ironhack,10828
1,Verified Outcomes,verified_outcomes,<p>School publishes a third-party verified out...,ironhack,10828
2,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,ironhack,10828
0,Available Online,available_online,<p>School offers fully online courses</p>,app-academy,10525
1,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,app-academy,10525
2,Job Guarantee,job_guarantee,<p>School guarantees job placement</p>,app-academy,10525
0,Available Online,available_online,<p>School offers fully online courses</p>,springboard,11035
1,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,springboard,11035
2,Job Guarantee,job_guarantee,<p>School guarantees job placement</p>,springboard,11035


In [29]:
#badges = badges[['school_id','school','keyword', 'name','description']].copy()
badges['description'] = badges['description'].str.replace(r'(?<=<).*?(?=>)', '', regex = True)
badges['description'] = badges['description'].str.replace('<>', '', regex = True)
badges.reset_index(inplace = True, drop = True)
badges.index += 1

badges

,name,keyword,description,school,school_id
1,Available Online,available_online,School offers fully online courses,ironhack,10828
2,Verified Outcomes,verified_outcomes,School publishes a third-party verified outcom...,ironhack,10828
3,Flexible Classes,flexible_classes,School offers part-time and evening classes,ironhack,10828
4,Available Online,available_online,School offers fully online courses,app-academy,10525
5,Flexible Classes,flexible_classes,School offers part-time and evening classes,app-academy,10525
6,Job Guarantee,job_guarantee,School guarantees job placement,app-academy,10525
7,Available Online,available_online,School offers fully online courses,springboard,11035
8,Flexible Classes,flexible_classes,School offers part-time and evening classes,springboard,11035
9,Job Guarantee,job_guarantee,School guarantees job placement,springboard,11035


Deleting some rows so that we have only unique values in 'name'

In [34]:
badges.drop(badges[badges['school_id'] == 10525].index, inplace=True)
badges.drop([7, 8], inplace=True)


badges

KeyError: '[7, 8] not found in axis'

In [36]:
badges.reset_index(inplace = True, drop = True)
badges.index += 1
badges

,name,keyword,description,school,school_id
1,Available Online,available_online,School offers fully online courses,ironhack,10828
2,Verified Outcomes,verified_outcomes,School publishes a third-party verified outcom...,ironhack,10828
3,Flexible Classes,flexible_classes,School offers part-time and evening classes,ironhack,10828
4,Job Guarantee,job_guarantee,School guarantees job placement,springboard,11035


Deleting columns we don`t need

In [39]:
badges.drop(columns=['keyword', 'school', 'school_id'], inplace=True)
badges

,name,description
1,Available Online,School offers fully online courses
2,Verified Outcomes,School publishes a third-party verified outcom...
3,Flexible Classes,School offers part-time and evening classes
4,Job Guarantee,School guarantees job placement


Adding new column as a copy of index and renaming it 'badge_id'

In [40]:
badges['index'] = badges.index
badges.rename(columns={'index': 'badge_id'}, inplace=True)
badges = badges.reindex(columns=['badge_id', 'name', 'description'])
badges

,badge_id,name,description
1,1,Available Online,School offers fully online courses
2,2,Verified Outcomes,School publishes a third-party verified outcom...
3,3,Flexible Classes,School offers part-time and evening classes
4,4,Job Guarantee,School guarantees job placement


saving to .csv file

In [42]:
badges.to_csv("CSV//badges.csv",index = False, sep=";")

# Data Cleaning Comments DF

Deleting and Renaming columns

In [43]:
comments.head(3)

,id,name,anonymous,hostProgramName,graduatingYear,isAlumni,jobTitle,tagline,body,rawBody,...,queryDate,program,user,overallScore,comments,overall,curriculum,jobSupport,review_body,school
0,306372,Sergio Burgos,False,None,2023.0,False,International Negotiator,The Most Intense Academic Challenge,"<span class=""truncatable""><p></p><p>After comp...",<p>After completing my Data Analytics Bootcamp...,...,2023-11-10,Data Analytics Bootcamp,{'image': None},3.3,[],3.0,3.0,4.0,After completing my Data Analytics Bootcamp wi...,ironhack
1,306215,Anonymous,True,None,2023.0,True,,Transformative Experience: My Time at Ironhack,"<span class=""truncatable""><p></p><p>Pros: 1)In...",<p>Pros: 1)Intensive Learning 2)Real-World Pro...,...,2023-11-06,Web Development Bootcamp,{'image': None},4.0,[],4.0,4.0,4.0,Pros: 1)Intensive Learning 2)Real-World Projec...,ironhack
2,306068,Anonymous,True,None,2023.0,False,Full stack development,Now I can do it,"<span class=""truncatable""><p></p><p>7 months a...","<p>7 months ago, I only had an idea about html...",...,2023-10-31,None,{'image': None},5.0,[],5.0,5.0,5.0,"7 months ago, I only had an idea about html an...",ironhack


In [ ]:
comments.columns

Creating a copy of the original DF only with columns we need to match the columns in the MySQL DB

In [44]:
comments_new=comments[['id', 'name', 'anonymous', 'graduatingYear',
       'isAlumni', 'jobTitle', 'tagline', 'createdAt', 'program', 'overallScore', 'overall',
       'curriculum', 'jobSupport', 'review_body', 'school']]

comments_new.head(2)

,id,name,anonymous,graduatingYear,isAlumni,jobTitle,tagline,createdAt,program,overallScore,overall,curriculum,jobSupport,review_body,school
0,306372,Sergio Burgos,False,2023.0,False,International Negotiator,The Most Intense Academic Challenge,11/10/2023,Data Analytics Bootcamp,3.3,3.0,3.0,4.0,After completing my Data Analytics Bootcamp wi...,ironhack
1,306215,Anonymous,True,2023.0,True,,Transformative Experience: My Time at Ironhack,11/6/2023,Web Development Bootcamp,4.0,4.0,4.0,4.0,Pros: 1)Intensive Learning 2)Real-World Projec...,ironhack


In [46]:
comments_new.rename(columns={'id':'comment_id', 'graduatingYear':'graduating_year',
       'isAlumni':'is_alumni', 'jobTitle':'job_title', 'tagline':'tag_line', 'createdAt':'date', 'program':'courses', 'overallScore':'overall_score', 'jobSupport':'job_support', 'review_body':'body'}, inplace=True)

comments_new.columns

C:\Users\Win10 Pro x64\AppData\Local\Temp\ipykernel_14148\620871338.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments_new.rename(columns={'id':'comment_id', 'graduatingYear':'graduating_year',


Index(['comment_id', 'name', 'anonymous', 'graduating_year', 'is_alumni',
       'job_title', 'tag_line', 'date', 'courses', 'overall_score', 'overall',
       'curriculum', 'job_support', 'body', 'school'],
      dtype='object')

reordering the columns so that they match with the tables in MySQL

In [47]:
comments_new = comments_new.reindex(columns=['comment_id', 'date', 'name', 'anonymous', 'graduating_year', 'is_alumni',
       'job_title', 'tag_line', 'body', 'courses', 'overall_score', 'overall',
       'curriculum', 'job_support', 'school'])

Creating a new column for student_id as a duplicate of the column 'name'

In [48]:
comments_new['student_id'] = comments_new.loc[:, 'name'] 


Assigning unique 5 digit values as student ids for every student name 
'Anonymous' = 10001

In [49]:
comments_new['student_id'] = pd.factorize(comments_new['student_id'])[0] + 10000


reordering again

In [50]:
comments_new = comments_new.reindex(columns=['comment_id', 'student_id', 'date', 'name', 'anonymous', 'graduating_year',
       'is_alumni', 'job_title', 'tag_line', 'body', 'courses',
       'overall_score', 'overall', 'curriculum', 'job_support', 'school'
       ])

## Checking for missing values

In [53]:
miss_val = pd.DataFrame(round(comments_new.isna().sum()/len(comments_new),4)*100)
miss_val = miss_val.reset_index()
miss_val.columns = ['column_name', 'percentage_of_missing_values']
display(miss_val)

,column_name,percentage_of_missing_values
0,comment_id,0.00
1,student_id,0.00
2,date,0.00
3,name,0.00
4,anonymous,0.00
5,graduating_year,0.10
6,is_alumni,0.00
7,job_title,35.20
8,tag_line,0.02
9,body,0.00


We can`t have missing values in the score columns, as we plan to use them for modelling, so we decided to drop those rows. 

In [54]:

comments_new.dropna(subset=['overall_score'], inplace=True)
comments_new.dropna(subset=['overall'], inplace=True)
comments_new.dropna(subset=['curriculum'], inplace=True)
comments_new.dropna(subset=['job_support'], inplace=True)
comments_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3758 entries, 0 to 1564
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   comment_id       3758 non-null   int64  
 1   student_id       3758 non-null   int64  
 2   date             3758 non-null   object 
 3   name             3758 non-null   object 
 4   anonymous        3758 non-null   bool   
 5   graduating_year  3754 non-null   float64
 6   is_alumni        3758 non-null   bool   
 7   job_title        2491 non-null   object 
 8   tag_line         3758 non-null   object 
 9   body             3758 non-null   object 
 10  courses          3556 non-null   object 
 11  overall_score    3758 non-null   object 
 12  overall          3758 non-null   object 
 13  curriculum       3758 non-null   object 
 14  job_support      3758 non-null   object 
 15  school           3758 non-null   object 
dtypes: bool(2), float64(1), int64(2), object(11)
memory usage: 4

Adding 'schoo_id' column and assingning it`s values by mapping from a dictionary with the school_ids

In [56]:
comments_new['school_id'] = comments_new.loc[:, 'school']
school_idS = {   
'ironhack' : 10828,
'app-academy' : 10525,
'springboard' : 11035    
}
comments_new['school_id'] = comments_new['school_id'].map(school_idS)


Adding 'course_id' column and assingning it`s values by mapping from a dictionary with the course_ids

In [59]:
comments_new['course_id'] = comments_new.loc[:, 'courses']
course_id_mapping=courses.set_index('course')['course_id'].to_dict()
comments_new['course_id'] = comments_new['course_id'].map(course_id_mapping)



,comment_id,student_id,date,name,anonymous,graduating_year,is_alumni,job_title,tag_line,body,courses,overall_score,overall,curriculum,job_support,school,school_id,course_id
0,306372,10000,11/10/2023,Sergio Burgos,False,2023.0,False,International Negotiator,The Most Intense Academic Challenge,After completing my Data Analytics Bootcamp wi...,Data Analytics Bootcamp,3.3,3.0,3.0,4.0,ironhack,10828,3.0
1,306215,10001,11/6/2023,Anonymous,True,2023.0,True,,Transformative Experience: My Time at Ironhack,Pros: 1)Intensive Learning 2)Real-World Projec...,Web Development Bootcamp,4.0,4.0,4.0,4.0,ironhack,10828,7.0
2,306068,10001,10/31/2023,Anonymous,True,2023.0,False,Full stack development,Now I can do it,"7 months ago, I only had an idea about html an...",None,5.0,5.0,5.0,5.0,ironhack,10828,NaN
3,305297,10002,10/2/2023,Utku Cikmaz,False,2023.0,False,Full Stack Web Developer,It was good,"The course was great. Especially, Luis is a gr...",Web Development Bootcamp,4.0,5.0,3.0,4.0,ironhack,10828,7.0
4,305278,10003,9/30/2023,Nirmal Hodge,False,2023.0,False,Product Designer,Ironhack 100% Worth It!,I joined the UX/ UI Bootcamp and to be honest ...,UX/UI Design Bootcamp,5.0,5.0,5.0,5.0,ironhack,10828,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1560,234894,13138,12/15/2016,Stephanie S.,False,2015.0,True,Website Manager-UX Lead,Excellent and comprehensive UX course,The UX Design Workshop gave me a great underst...,UX Design,5.0,5.0,5.0,5.0,springboard,11035,NaN
1561,234877,13139,12/15/2016,Joe Fang,False,2015.0,True,UX Designer,Springboard helped me start my career in UX,"Just some background information on myself, I ...",UX Design,5.0,5.0,5.0,5.0,springboard,11035,NaN
1562,234838,13140,12/15/2016,Zeina,False,2016.0,True,Senior Graphic Designer,Highly recommended,"I enjoyed the program very much, gained a lot ...",UX Design,3.7,4.0,4.0,3.0,springboard,11035,NaN
1563,242681,13141,12/12/2016,Jean,False,2016.0,True,Lead Designer/Creative Director,A way to stay current for the highly motivated!,This was my first online experience and it did...,UX Design,5.0,5.0,5.0,5.0,springboard,11035,NaN


In [63]:
comments_new.head(3)

,comment_id,student_id,date,name,anonymous,graduating_year,is_alumni,job_title,tag_line,body,courses,overall_score,overall,curriculum,job_support,school,school_id,course_id
0,306372,10000,11/10/2023,Sergio Burgos,False,2023.0,False,International Negotiator,The Most Intense Academic Challenge,After completing my Data Analytics Bootcamp wi...,Data Analytics Bootcamp,3.3,3.0,3.0,4.0,ironhack,10828,3.0
1,306215,10001,11/6/2023,Anonymous,True,2023.0,True,,Transformative Experience: My Time at Ironhack,Pros: 1)Intensive Learning 2)Real-World Projec...,Web Development Bootcamp,4.0,4.0,4.0,4.0,ironhack,10828,7.0
2,306068,10001,10/31/2023,Anonymous,True,2023.0,False,Full stack development,Now I can do it,"7 months ago, I only had an idea about html an...",None,5.0,5.0,5.0,5.0,ironhack,10828,NaN


In [71]:
comments_new['graduating_year'] = pd.to_numeric(comments_new['graduating_year'], errors='coerce').astype('Int64')

Creating a .csv file

In [75]:
comments_new.to_csv("CSV/comments_new.csv",index = False, sep=";")

creating a new 'students' DF as a copy of the needed columns from the comments DF

In [76]:
students=comments_new[['student_id', 'name', 'job_title', 'graduating_year', 'school_id', 'course_id']]
students.head(3)

,student_id,name,job_title,graduating_year,school_id,course_id
0,10000,Sergio Burgos,International Negotiator,2023,10828,3.0
1,10001,Anonymous,,2023,10828,7.0
2,10001,Anonymous,Full stack development,2023,10828,NaN


Deleting the 'anonymous' students and dropping duplicates

In [77]:
students = students.drop(students[students['student_id'] == 10001].index)
students = students.drop_duplicates(subset='student_id', keep='first')

creating the students_courses table and saving as csv

In [84]:
students_courses = students[['student_id', 'course_id']]
students_courses

,student_id,course_id
0,10000,3.0
3,10002,7.0
4,10003,5.0
7,10004,4.0
13,10009,7.0
...,...,...
1559,13137,NaN
1560,13138,NaN
1561,13139,NaN
1562,13140,NaN


In [85]:
students_courses.to_csv("CSV/students_courses.csv",index = False, sep=";")

In [87]:
students.drop(columns=['course_id'], inplace=True)
students

,student_id,name,job_title,graduating_year,school_id
0,10000,Sergio Burgos,International Negotiator,2023,10828
3,10002,Utku Cikmaz,Full Stack Web Developer,2023,10828
4,10003,Nirmal Hodge,Product Designer,2023,10828
7,10004,Sabir Karimov,Data Analyst,2023,10828
13,10009,Mercedes Amor Gallart,,2023,10828
...,...,...,...,...,...
1559,13137,Jenny N.,None,2016,11035
1560,13138,Stephanie S.,Website Manager-UX Lead,2015,11035
1561,13139,Joe Fang,UX Designer,2015,11035
1562,13140,Zeina,Senior Graphic Designer,2016,11035


In [88]:
students.to_csv("CSV/students.csv",index = False, sep=";")